In [61]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import math
import os

video_dir = "E:/研究生期间学习/Tencent/psnr"
x264 = "/x264"
cbr = "/cbr"
abr = "/abr"

mask = "/mask"
# raw = "/raw"

x264s = [video_dir + x264 + "/" + f for f in os.listdir(video_dir + x264) if f.endswith((".jpg", ".jpeg", ".png"))]
cbrs = [video_dir + cbr + "/" + f for f in os.listdir(video_dir + cbr) if f.endswith((".jpg", ".jpeg", ".png"))]
abrs = [video_dir + abr + "/" + f for f in os.listdir(video_dir + abr) if f.endswith((".jpg", ".jpeg", ".png"))]
masks = [video_dir + mask + "/" + f for f in os.listdir(video_dir + mask) if f.endswith((".jpg", ".jpeg", ".png"))]

pattern = {"cbr": cbrs, "abr": abrs}

mask_locs = {}

def mask_location(masks):
    for mask in masks:
        image_mask = np.array(Image.open(mask).convert('L'))
        binar_mask = image_mask < 125
#        image_mask[binar_mask] = 0
        
        idx = mask.split('/')
        idx = idx[-1]
        idx = idx[0:4]

        
        mask_locs[idx] = binar_mask


def ave_psnr(mask_locs, state):
    images = pattern[state]
    images1 = []
    images2 = []
    ave_psnr = 0
    
    index = 1
    for image in images:
        if index > len(mask_locs): break

        image1 = np.array(Image.open(image).convert('L'))
        
        idx = image.split('/')
        idx = idx[-1]
        idx = idx[0:4]
        
        image1[mask_locs[idx]] = 0
        images1.append(image1)

        index = index + 1


    index = 1               
    for x264 in x264s:
        if index > len(mask_locs): break

        image2 = np.array(Image.open(x264).convert('L'))
               
        idx = x264.split('/')
        idx = idx[-1]
        idx = idx[0:4]
               
        image2[mask_locs[idx]] = 0
        images2.append(image2)

        index = index + 1

    for img1, img2 in zip(images1, images2):
            temp = psnr(img1, img2)
            ave_psnr = ave_psnr + temp
    ave_psnr = ave_psnr / len(images1)

    return ave_psnr

    
def psnr(img1, img2):
   mse = np.mean((img1/1.0 - img2/1.0) ** 2 )
   if mse < 1.0e-10:
      return 100
   return 10 * math.log10(255.0**2/mse)

In [66]:
mask_location(masks)
ave_psnr(mask_locs, state="cbr")

34.62866191250379

In [69]:
mask_location(masks)
ave_psnr(mask_locs, state="abr")

40.37813576936062